In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain_community.llms.ollama import Ollama
import streamlit as st
from datetime import datetime
import mysql.connector
import torch
import time
import csv


In [2]:

#Ollama
llm = Ollama(model="llama3.1",temperature=0)
# Load the tokenizer and model for mental health sentiment analysis
tokenizer = AutoTokenizer.from_pretrained("tahaenesaslanturk/mental-health-classification-v0.1")
model = AutoModelForSequenceClassification.from_pretrained("tahaenesaslanturk/mental-health-classification-v0.1")



C:\Users\Suvra\AppData\Local\Temp\ipykernel_41620\3004090961.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1",temperature=0)


In [3]:
#Analyze the sentiment
#Function to check if context suggests a potential mental condition
def sentiment_present(content:str):
    #If content is too short, return false
    if(len(content.split())<4):
        return False


    prompt = '''Analyze the following message from a student seeking mental assistance.
    Respond with 'yes' if the message indicates a potential mental or physical health concern.
    Otherwise, respond with 'no'. Do not provide any additional text. 
    User input:''' 
    prompt += content

    full_response = ""
    response = llm.stream(prompt)

    max_size = 8
    size = 0
    for token in response:
        full_response += token
        size+=1
        if(size>max_size):
            break
    #Get the first word
    token = full_response.split()[0]
    
    #Only return 0 if we are sure there is no issue
    if(token.lower()=="no"):
        return False
    else:
        return True

#Analyze the sentiment
def sentiment(content:str):
    # Encode the input text
    inputs = tokenizer(content, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    label = model.config.id2label[predicted_label]
    return label

#A function to format the prediction
def format_prediction(prediction):
    match(prediction):
        case "EDAnonymous":
            return "eating disorder"
        case "bipolarreddit":
            return "bipolar disorder"
        case "bpd":
            return "bipolar disorder"
        case "healthanxiety":
            return "anxiety"
        case "socialanxiety":
            return "anxiety"
        case "lonely":
            return "loneliness"
        case "ptsd":
            return "PTSD"
        case "alcoholism":
            return "emotional"
        case "suicidewatch":
            return "extreme"
        case _:
            return prediction

In [ ]:

disorder = 0

accuracy =0 
id =0


with open("data.csv",newline='\n') as csvfile:
    rows = csv.reader(csvfile,delimiter=',')
    for row in rows:
        content = row[0]

        #List of issues
        secondary_labels = []
        
        for i in range(1,len(row)):
            secondary_labels.append(row[i].lower().strip())
            

        #Check if we can find an issue
        shouldCheck = sentiment_present(content)

        
        prediction = "normal"
        if(shouldCheck):
            prediction = sentiment(content)

        prediction = format_prediction(prediction)
        id+=1
        print(id," ",content,end="")
        print(",",secondary_labels,":",prediction)

        #Check whether it can 
        if("normal" in secondary_labels and prediction.lower().strip() == "normal" or "normal" not in secondary_labels and prediction.lower().strip() != "normal"):
            disorder+=1
            print("Presence or absence of disorder detected correctly")
       
        if(prediction.lower().strip() in secondary_labels):
            accuracy+=1
            print("Classification matched correctly")
        
        

print("Accuracy of classification = ",(accuracy/id)*100)
print("Accuracy of finding disorder = ",(disorder/id)*100)

1   I had a great day at work and met some new friends., ['normal', '', ''] : normal
Presence or absence of disorder detected correctly
Classification matched correctly
2   Today was just an average day  nothing special., ['normal', '', ''] : normal
Presence or absence of disorder detected correctly
Classification matched correctly
3   I enjoy going for morning walks to start my day fresh., ['normal', '', ''] : normal
Presence or absence of disorder detected correctly
Classification matched correctly
4   Cooking is my favorite way to relax in the evenings., ['normal', '', ''] : normal
Presence or absence of disorder detected correctly
Classification matched correctly
5   I feel guilty every time I eat something unhealthy., ['eating disorder', '', ''] : eating disorder
Presence or absence of disorder detected correctly
Classification matched correctly
6   No matter how much I eat  I never feel satisfied., ['eating disorder', '', ''] : eating disorder
Presence or absence of disorder dete